<div style="widelta_th: 100%; overflow: hidden;">
    <div style="display: block;margin-left: auto;margin-right: auto; widelta_th: 100%;"><img src="./logo_dei.png"  /></div>
</div>

<h2 style="font-family:Avantgarde; font-size:2em; text-align: center;">Projeto 1 - Fusão de Informação e Análise de Dados</h2>


<h4 style="font-family:Avantgarde; font-size:1.75em; text-align: center;">Mestrado em Engenharia e Ciência de Dados</h4>
<h4 style="font-family:Avantgarde; font-size:1.3em; text-align: center;">Ano Letivo 2023/2024</h4>

<p style = "font-family:Avantgarde; font-size:1.1em; text-align: center;"> <b>Mariana Lopes Paulino - 2020190448</b></p>
<p style = "font-family:Avantgarde; font-size:1.1em; text-align: center;"> <b>Rui Alexandre Coelho Tapadinhas - 2018283200</b></p>

<p style="font-family:Avantgarde; font-size:1.25em;"> <b> Tarefas </b> </p>
<p style="font-family:Avantgarde; font-size:1em;"> 1. Criar uma função que implementa o modelo dinâmico do robot; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 2. Criar uma função que implementa o modelo de observação; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 3. Criar uma função que representa a aquisição de dados dos sensores; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 4. Criar uma função que implementa o Extended Kalman Filter (EKF); </p>
<p style="font-family:Avantgarde; font-size:1em;"> 5. Criar uma função que implementa o Unscented Kalman Filter (UKF); </p>
<p style="font-family:Avantgarde; font-size:1em;"> 6. Especificar os vários parâmetros dos filtros e de simulação do robot; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 7. Simular o movimento do robot e representar dinamicamente a sua localização, considerando as trajetórias e a elipse indicadas na Figura 1. </p>


<p style="font-family:Avantgarde; font-size:1.15em;"> <b> Informações Relevantes </b> </p>
<p style="font-family:Avantgarde; font-size:1em;"> O robot tem 4 estados no instante k. xk, yk, Φk, vt.  (x, y) é a posição 2D do robot. Φ é a sua orientação e v a sua velocidade. </p>

<p style="font-family:Avantgarde; font-size:1.5em;"> 0. Import Libraries </p>

In [1]:
import kaleido
import imageio
from math import sqrt, cos, sin, ceil, atan2, pi
import matplotlib.pyplot as plt
import numpy as np
from plotly import graph_objects as go
import pandas as pd
import os
from scipy.linalg import sqrtm, inv
from sklearn.metrics import mean_squared_error as mse, r2_score as r2

<p style="font-family:Avantgarde; font-size:1.5em;"> 
1. Parâmetros dos Filtros e da Simulação do Robot </p>

<p style="font-family:Avantgarde; font-size:1em;"> Nesta primeira secção definimos os parâmetros que vão ser utilizadps globalmente mais tarde nos filtros e na simulação da localização do robot. Na variável Q é definida a variância no eixo do x, no eixo do y, yaw e velocidade. Nesta secção é ainda definido o intervalo de tempo Δt, a velocidade v (m/s), a velocidade angular Φ (rad/s) e o tempo (s) que desejamos para a nossa simulação. Todos estes dados irão ser usados mais tarde uma vez que são parâmetros decisivos para o robot.</p>

In [2]:
# Covariância para Simular UKF
Q = np.diag([0.1, 0.1, np.deg2rad(1.0), 1.0]) ** 2  
# Variância no eixo do x, eixo do y, yaw, velocidade

R = np.diag([1.0, 1.0]) ** 2
# Observar a covariância da posição x,y  

# Parâmetros da Simulação
# Intervalo de Tempo [s]
delta_t = 0.1  

# Velocidade medida em [m/s]
v = 1.0  

# Velocidade angular medida em [rad/s]
yawrate = 0.1  

# Tempo de Simulação [s]
sim_time = 50.0  

H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]], dtype=float)

#  Parametros para Filtro UKF
ALPHA = 0.001 #Dispersão dos sigma points
BETA = 2 #Distribuição do ruido (random)
KAPPA = 0 #Influencia a ponderação dos pontos sigma, média e covariância previstas

<p style="font-family:Avantgarde; font-size:1.5em;"> 2. Aquisição de Dados dos Sensores </p>

<p style="font-family:Avantgarde; font-size:1em;"> Nesta etapa do projeto vamos receber o vetor u que vai ter como dados a velocidade e a orientação do robot no instante k. Após a receção desses dados, é necessária a adição de ruído tanto ao GPS como aos dados recebidos como Inputs do instante k em que nos encontramos.</p>

In [3]:
def input():
    u = np.array([[v, yawrate]]).T
    return u

In [4]:
# Ruídos Aplicados

GPS_Noise = np.diag([0.5, 0.5])**2
Input_Noise = np.diag([1.0, np.deg2rad(30.0)])**2

In [5]:
def observation(xTrue, xd, u):

    xTrue = dynamic_motion_model(xTrue, u)

    # add noise to GPS x-y
    zx = xTrue[0, 0] + np.random.randn() * GPS_Noise[0, 0]
    zy = xTrue[1, 0] + np.random.randn() * GPS_Noise[1, 1]
    z = np.array([[zx, zy]])

    # add noise to input
    ud1 = u[0, 0] + np.random.randn() * Input_Noise[0, 0]
    ud2 = u[1, 0] + np.random.randn() * Input_Noise[1, 1]
    ud = np.array([[ud1, ud2]]).T

    xd = dynamic_motion_model(xd, ud)

    return xTrue, z, xd, ud

<p style="font-family:Avantgarde; font-size:1.5em;"> 3. Modelo Dinâmico </p>

<p style="font-family:Avantgarde; font-size:1em;"> O modelo não linear dinâmico do robot é definido pela expressão: </p>

<p style="font-family:Avantgarde; font-size:1em; text-align: center;"> <b> x(k+1) = A*x(k) + B*u(k) + w(k) </b> </p>

<p style="font-family:Avantgarde; font-size:1em;"> onde x(k) = [x, y, Φ, v] é o vetor de estados, u(k) = [v, Φ] é o vetor de inputs, A é a matriz de transição de estados, B é a matriz de inputs e w(k) é o ruído do processo. </p>


<p style="font-family:Avantgarde; font-size:1em;"> Neste modelo, vai ser criada uma matriz com a dimensão 4x4 onde os elementos da diagonal principal irão ser 1, 1, 1 e 0 respetivamente. Tal como definido em A. </p>

In [6]:
def dynamic_motion_model(x, u):
    yaw = x[2,0]

    A = np.diag([1.0, 1.0, 1.0, 0])
    #print(A)

    B = np.array([[cos(yaw) * delta_t, 0],
                  [sin(yaw) * delta_t, 0],
                  [0.0, delta_t],
                  [1.0, 0.0]])
    #print(B)

    x = A @ x + B @ u

    return x

<p style="font-family:Avantgarde; font-size:1.5em;"> 4. Modelo de Observação </p>

<p style="font-family:Avantgarde; font-size:1em;"> O robot vai seguir um modelo de observação cuja fórmula é dada por: </p>

<p style="font-family:Avantgarde; font-size:1em; text-align: center;"> <b> z(k) = C*x(k) + r(k) </b> </p>

In [7]:
def observation_model(x):

    C = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0]])

    z = C @ x

    return z

In [8]:
def jacobianaA(x, u):
    v = u[0, 0]
    yaw = x[2,0]
    
    jacobA = np.array([[1, 0, -v * np.sin(yaw) * delta_t, np.cos(yaw) * delta_t],
                   [0, 1, v * np.cos(yaw) * delta_t, np.sin(yaw) * delta_t],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]], dtype=float)
    
    return jacobA

In [9]:
def jacobianaC():
    jacobC = np.array([[1, 0, 0, 0],
                   [0, 1, 0, 0]], dtype=float)

    return jacobC

In [10]:
def obs_model(x):
    return jacobianaC().dot(x)

<p style="font-family:Avantgarde; font-size:1em;"> Ambos modelos tanto de Observação como Dinâmico foram implementados com base nas fichas práticas e nos dados do enunciado do projeto. </p>

<p style="font-family:Avantgarde; font-size:1.5em;"> 5. Extended Kalman Filter (EKF) </p>

In [11]:
def ekf(x, p, z, u):
    # Predict
    xpred = dynamic_motion_model(x, u)
    jA = jacobianaA(xpred, u)
    ppred = jA.dot(p).dot(jA.T) + Q

    # Atualizar
    jC = jacobianaC()
    zpred = obs_model(xpred)
    y = z.T - zpred
    S = jC.dot(ppred).dot(jC.T) + R
    K = ppred.dot(jC.T).dot(np.linalg.inv(S))
    x = xpred + K.dot(y)
    p = (np.eye(len(x)) - K.dot(jC)).dot(ppred)

    return x, p

In [12]:
def save(hxpred, xpred, hxtrue, xtrue, hxdr, xdr, hz, z):
    hxpred = np.hstack((hxpred, xpred))
    hxtrue = np.hstack((hxtrue, xtrue))
    hxdr = np.hstack((hxdr, xdr))
    hz = np.vstack((hz, z))
    return hxpred, hxtrue, hxdr, hz

In [13]:
def plot_covariance_ellipse(xEst, PEst):
    Pxy = PEst[0:2, 0:2]
    eigval, eigvec = np.linalg.eig(Pxy)

    if eigval[0] >= eigval[1]:
        bigind = 0
        smallind = 1
    else:
        bigind = 1
        smallind = 0

    t = np.arange(0, 2 * pi + 0.1, 0.1)
    a = sqrt(eigval[bigind])
    b = sqrt(eigval[smallind])
    x = [a * cos(it) for it in t]
    y = [b * sin(it) for it in t]
    angle = atan2(eigvec[bigind, 1], eigvec[bigind, 0])
    R = np.array([[cos(angle), sin(angle)],
                  [-sin(angle), cos(angle)]])
    fx = R.dot(np.array([[x, y]]))
    px = np.array(fx[0, :] + xEst[0, 0]).flatten()
    py = np.array(fx[1, :] + xEst[1, 0]).flatten()

    return px, py

In [14]:

#def main():
print('Simulação do Filtro EKF ...')


# Inicializar o tempo a 0
time = 0.

# vetor de estados [x, y, yaw, v]
xpred_ekf = np.zeros((4, 1))    
xtrue_ekf = np.zeros((4, 1))    
ppred_ekf = np.eye(4)

# dead reckoning
xdr_ekf = np.zeros((4, 1))

# histórico
hxpred_ekf = xpred_ekf
hxtrue_ekf = xtrue_ekf
hxdr_ekf = xtrue_ekf
hz_ekf = np.zeros((1, 2))

fig = plt.figure()
fig.show()

fig = plt.figure()
fig.show()

while time <= sim_time:
    time += delta_t

    xtrue_ekf, z, xdr_ekf, ud = observation(xtrue_ekf, xdr_ekf, input())
    xpred_ekf, ppred_ekf = ekf(xpred_ekf, ppred_ekf, z, ud)
    hxpred_ekf = np.hstack((hxpred_ekf, xpred_ekf))
    hxdr_ekf = np.hstack((hxdr_ekf, xdr_ekf))
    hxtrue_ekf = np.hstack((hxtrue_ekf, xtrue_ekf))
    hz_ekf = np.vstack((hz_ekf, z))

# End
print("Ultimo instante")

# Elipse de Covariância
px, py = plot_covariance_ellipse(xpred_ekf, ppred_ekf)
ellipse_trace_ekf = (go.Scatter(x=px, y=py, mode='lines', line=dict(color='red', dash='dash')))


# Criar Traços
trace_observation_ekf = go.Scatter(x=hz_ekf[:, 0], y=hz_ekf[:, 1], mode='markers', name='Observação', marker=dict(color='green', size=8))
trace_true_ekf = go.Scatter(x=hxtrue_ekf[0, :].flatten(), y=hxtrue_ekf[1, :].flatten(), mode='lines', name='Verdadeiro', line=dict(color='blue'))
trace_dead_reckoning_ekf = go.Scatter(x=hxdr_ekf[0, :].flatten(), y=hxdr_ekf[1, :].flatten(), mode='lines', name='Dead Reckoning', line=dict(color='black'))
trace_ekf_prediction_ekf = go.Scatter(x=hxpred_ekf[0, :].flatten(), y=hxpred_ekf[1, :].flatten(), mode='lines', name='Predição EKF', line=dict(color='red'))

# Lista de Traços do Gráfico
data = [trace_observation_ekf, trace_true_ekf, trace_dead_reckoning_ekf, trace_ekf_prediction_ekf, ellipse_trace_ekf]

# Configurar o Layout
layout = go.Layout(
    title='Gráfico da Posição do Robot - EKF',
    legend=dict(x=1, y=1),
    autosize=False,
    width=800,
    height=600,
    margin=dict(l=40, r=40, b=40, t=40)
)

# Criar figura
fig = go.Figure(data=data, layout=layout)

fig.show()

filename = f'ekf_{int(ceil(time * 10))}.jpeg'
fig.write_image(filename)


comment = """
        # Criar Traços
        trace_observation = go.Scatter(x=hz[:, 0], y=hz[:, 1], mode='markers', name='Observação', marker=dict(color='green', size=8))
        trace_true = go.Scatter(x=hxtrue[0, :].flatten(), y=hxtrue[1, :].flatten(), mode='lines', name='Verdadeiro', line=dict(color='blue'))
        trace_dead_reckoning = go.Scatter(x=hxdr[0, :].flatten(), y=hxdr[1, :].flatten(), mode='lines', name='Dead Reckoning', line=dict(color='black'))
        trace_ekf_prediction = go.Scatter(x=hxpred[0, :].flatten(), y=hxpred[1, :].flatten(), mode='lines', name='Predição EKF', line=dict(color='red'))

        # Lista de Traços do Gráfico
        data = [trace_observation, trace_true, trace_dead_reckoning, trace_ekf_prediction]

        # Configurar o Layout
        layout = go.Layout(
            title='Gráfico da Posição do Robot - EKF',
            legend=dict(x=1, y=1),
            autosize=False,
            width=800,
            height=600,
            margin=dict(l=40, r=40, b=40, t=40)
        )

        # Criar figura
        fig = go.Figure(data=data, layout=layout)

        # Frames EKF
        if not os.path.exists('frames_ekf'):
            os.makedirs('frames_ekf')

        filename = f'frames_ekf/{int(ceil(time * 10))}.jpeg'
        fig.write_image(filename)


    # Fazer uma Animação em GIF
    images = []
    filenames = os.listdir('frames_ekf')
    filenames.sort(key=lambda x: int(x.split('.')[0]))
    for filename in filenames:
        images.append(imageio.imread(f'frames_ekf/{filename}'))
    imageio.mimsave('ekf.gif', images, duration=0.1)"""

Simulação do Filtro EKF ...
Ultimo instante


/var/folders/s8/ryzdv1sn1g15nt9h4886lz7m0000gn/T/ipykernel_36520/1383960505.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()
/var/folders/s8/ryzdv1sn1g15nt9h4886lz7m0000gn/T/ipykernel_36520/1383960505.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [15]:
#main()

<p style="font-family:Avantgarde; font-size:1em;"> Na secção acima desenvolvemos o Filtro Extended Kalman, guardámos todas as suas iterações dentro dos 50 segundos de tempo de simulação na pasta frames. Assim, conseguimos acompanhar o desenvolvimento da previsão do filtro a cada iteração k que passa. </p>

<p style="font-family:Avantgarde; font-size:1.5em;"> 6. Unscented Kalman Filter (UKF)</p>

In [16]:
def generate_sigma_points(xEst, PEst, gamma):
    sigma = xEst
    Psqrt = sqrtm(PEst)
    n = len(xEst[:, 0])
    # Positive direction
    for i in range(n):
        sigma = np.hstack((sigma, xEst + gamma * Psqrt[:, i:i + 1]))

    # Negative direction
    for i in range(n):
        sigma = np.hstack((sigma, xEst - gamma * Psqrt[:, i:i + 1]))

    return sigma

In [17]:
def predict_sigma_motion(sigma, u):
    #Sigma Points prediction with motion model
    
    for i in range(sigma.shape[1]):
        sigma[:, i:i + 1] = dynamic_motion_model(sigma[:, i:i + 1], u)

    return sigma

In [18]:
def predict_sigma_observation(sigma):
    #Sigma Points prediction with observation model
    
    for i in range(sigma.shape[1]):
        sigma[0:2, i] = observation_model(sigma[:, i])

    sigma = sigma[0:2, :]

    return sigma

In [19]:
def calc_sigma_covariance(x, sigma, wc, Pi):
    nSigma = sigma.shape[1]
    d = sigma - x[0:sigma.shape[0]]
    P = Pi
    for i in range(nSigma):
        P = P + wc[0, i] * d[:, i:i + 1] @ d[:, i:i + 1].T
    #print(P)
    return P

In [20]:
def calc_pxz(sigma, x, z_sigma, zb, wc):
    nSigma = sigma.shape[1]
    dx = sigma - x
    dz = z_sigma - zb[0:2]
    P = np.zeros((dx.shape[0], dz.shape[0]))

    for i in range(nSigma):
        P = P + wc[0, i] * dx[:, i:i + 1] @ dz[:, i:i + 1].T
    
    return P

In [21]:
def ukf_estimation(xEst, PEst, z, u, wm, wc, gamma):
    #  Predict
    sigma = generate_sigma_points(xEst, PEst, gamma)
    sigma = predict_sigma_motion(sigma, u)
    xPred = (wm @ sigma.T).T
    PPred = calc_sigma_covariance(xPred, sigma, wc, Q)

    #  Update
    zPred = observation_model(xPred)
    y = z - zPred
    sigma = generate_sigma_points(xPred, PPred, gamma)
    zb = (wm @ sigma.T).T
    
    z_sigma = predict_sigma_observation(sigma)
    st = calc_sigma_covariance(zb, z_sigma, wc, R)
    Pxz = calc_pxz(sigma, xPred, z_sigma, zb, wc)
    K = Pxz @ np.linalg.inv(st)
    xEst = xPred + K @ y
    PEst = PPred - K @ st @ K.T

    return xEst, PEst

In [22]:
def plot_covariance_ellipse(xEst, PEst):
    Pxy = PEst[0:2, 0:2]
    eigval, eigvec = np.linalg.eig(Pxy)

    if eigval[0] >= eigval[1]:
        bigind = 0
        smallind = 1
    else:
        bigind = 1
        smallind = 0

    t = np.arange(0, 2 * pi + 0.1, 0.1)
    a = sqrt(eigval[bigind])
    b = sqrt(eigval[smallind])
    x = [a * cos(it) for it in t]
    y = [b * sin(it) for it in t]
    angle = atan2(eigvec[bigind, 1], eigvec[bigind, 0])
    R = np.array([[cos(angle), sin(angle)],
                  [-sin(angle), cos(angle)]])
    fx = R.dot(np.array([[x, y]]))
    px = np.array(fx[0, :] + xEst[0, 0]).flatten()
    py = np.array(fx[1, :] + xEst[1, 0]).flatten()

    return px, py

In [23]:
def setup_ukf(nx):
    lamb = ALPHA ** 2 * (nx + KAPPA) - nx
    # calculate weights
    wm = [lamb / (lamb + nx)]
    wc = [(lamb / (lamb + nx)) + (1 - ALPHA ** 2 + BETA)]
    for i in range(2 * nx):
        wm.append(1.0 / (2 * (nx + lamb)))
        wc.append(1.0 / (2 * (nx + lamb)))
    gamma = sqrt(nx + lamb)

    wm = np.array([wm])
    wc = np.array([wc])

    return wm, wc, gamma

In [24]:

#def main_ukf():
print('Simulação do Filtro UKF ...')

nx = 4  # State Vector [x y yaw v]'

xEst_ukf = np.zeros((nx, 1))
xTrue_ukf = np.zeros((nx, 1))
PEst_ukf = np.eye(nx)

wm, wc, gamma = setup_ukf(nx)

xDR_ukf = np.zeros((nx, 1))  # Dead reckoning

# history
hxEst_ukf = xEst_ukf
hxTrue_ukf = xTrue_ukf
hxDR_ukf = xTrue_ukf
hz_ukf = np.zeros((2,1))

time = 0.0

fig = plt.figure()
fig.show()
SIM_TIME=50.0
while SIM_TIME >= time:
    time += delta_t
    u = input()

    xTrue_ukf, z, xDR_ukf, ud = observation(xTrue_ukf, xDR_ukf, u)
    z = z.T

    xEst_ukf, PEst_ukf = ukf_estimation(xEst_ukf, PEst_ukf, z, ud, wm, wc, gamma)
    #print(xEst)
    #print(PEst)

    # store data history
    hxEst_ukf = np.hstack((hxEst_ukf, xEst_ukf))
    hxDR_ukf = np.hstack((hxDR_ukf, xDR_ukf))
    hxTrue_ukf = np.hstack((hxTrue_ukf, xTrue_ukf))
    #print("hz_ukf", hz_ukf)
    hz_ukf = np.hstack((hz_ukf, z))

# Elipse de Covariância
px, py = plot_covariance_ellipse(xEst_ukf, PEst_ukf)
ellipse_trace_ukf = (go.Scatter(x=px, y=py, mode='lines', line=dict(color='red', dash='dash')))

# Criar Traços
trace_observation_ukf = go.Scatter(x=hz_ukf[0, :], y=hz_ukf[1, :], mode='markers', name='Observação', marker=dict(color='green', size=8))
trace_true_ukf = go.Scatter(x=hxTrue_ukf[0, :].flatten(), y=hxTrue_ukf[1, :].flatten(), mode='lines', name='Verdadeiro', line=dict(color='blue'))
trace_dead_reckoning_ukf = go.Scatter(x=hxDR_ukf[0, :].flatten(), y=hxDR_ukf[1, :].flatten(), mode='lines', name='Dead Reckoning', line=dict(color='black'))
trace_ukf_prediction_ukf = go.Scatter(x=hxEst_ukf[0, :].flatten(), y=hxEst_ukf[1, :].flatten(), mode='lines', name='Predição UKF', line=dict(color='red'))

# Lista de Traços do Gráfico
data = [trace_observation_ukf, trace_true_ukf, trace_dead_reckoning_ukf, trace_ukf_prediction_ukf, ellipse_trace_ukf]

# Configurar o Layout
layout = go.Layout(
    title='Gráfico da Posição do Robot - UKF',
    legend=dict(x=1, y=1),
    autosize=False,
    width=800,
    height=600,
    margin=dict(l=40, r=40, b=40, t=40)
)

# Criar figura
fig = go.Figure(data=data, layout=layout)

fig.show()

filename = f'ukf{int(ceil(time * 10))}.jpeg'
fig.write_image(filename)

comment = """
        # Criar Traços
        trace_observation = go.Scatter(x=hz[0, :], y=hz[1, :], mode='markers', name='Observação', marker=dict(color='green', size=8))
        trace_true = go.Scatter(x=hxTrue[0, :].flatten(), y=hxTrue[1, :].flatten(), mode='lines', name='Verdadeiro', line=dict(color='blue'))
        trace_dead_reckoning = go.Scatter(x=hxDR[0, :].flatten(), y=hxDR[1, :].flatten(), mode='lines', name='Dead Reckoning', line=dict(color='black'))
        trace_ukf_prediction = go.Scatter(x=hxEst[0, :].flatten(), y=hxEst[1, :].flatten(), mode='lines', name='Predição UKF', line=dict(color='red'))

        # Lista de Traços do Gráfico
        data = [trace_observation, trace_true, trace_dead_reckoning, trace_ukf_prediction]

        # Configurar o Layout
        layout = go.Layout(
            title='Gráfico da Posição do Robot - UKF',
            legend=dict(x=1, y=1),
            autosize=False,
            width=800,
            height=600,
            margin=dict(l=40, r=40, b=40, t=40)
        )

        # Criar figura
        fig = go.Figure(data=data, layout=layout)

        # Frames EKF
        if not os.path.exists('frames_ukf'):
            os.makedirs('frames_ukf')

        filename = f'frames_ukf/{int(ceil(time * 10))}.jpeg'
        fig.write_image(filename)


    # Fazer uma Animação em GIF
    images = []
    filenames = os.listdir('frames_ukf')
    filenames.sort(key=lambda x: int(x.split('.')[0]))
    for filename in filenames:
        images.append(imageio.imread(f'frames_ukf/{filename}'))
    imageio.mimsave('ukf.gif', images, duration=0.1)"""



Simulação do Filtro UKF ...


/var/folders/s8/ryzdv1sn1g15nt9h4886lz7m0000gn/T/ipykernel_36520/426314460.py:23: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



<Figure size 640x480 with 0 Axes>

In [25]:
#main_ukf()

<p style="font-family:Avantgarde; font-size:1.5em;"> 7. Simulação do Movimento do Robot </p>

<p style="font-family:Avantgarde; font-size:1em;"> Após a aplicação das fórmulas de ambos os filtros, obtemos os seguintes resultados: A primeira imagem corresponde a 50 segundos de simulação da posição do robot com recurso ao filtro EKF e a segunda com recurso ao filtro UKF também ela com 50 segundos de simulação.</p>

<p style="font-family:Avantgarde; font-size:1em;"> Em ambos os gráficos é apresentada uma linha a preto formada pelo Dead Reckoning, uma a azul escuro que representa o movimento verdadeiro do robot, uma linha a vermelho que representa a predição do filtro (seja este EKF no primeiro caso ou UKF no segundo), e por fim os marcadores verdes que fazem parte da Observação por parte do robot.

<p style="font-family:Avantgarde; font-size:1em;"> Antes da análise dos resultados do erro, gostávamos de realçar que o Dead Reckoning vai ser o que vai obter piores resultados uma vez que acumula erros das iterações passadas dando assim um valor superior na teoria. Isto acontece uma vez que ele utiliza a posição anterior do robot para calcular a seguinte.

<p style="font-family:Avantgarde; font-size:1em;"> Para o filtro EKF esperamos que o erro seja menor que o Dead Reckoning mas ao mesmo tempo superior que o do UKF uma vez que este filtro é uma versão não linear ao filtro de Kalman e, por isso é aplicado em sistemas não lineares. O primeiro passo para o seu funcionamento é o cálculo das matrizes de Jacobi que, no fundo são as derivadas parciais das funções dos estados (posições, orientação e velocidade). Partindo dessas matrizes para a previsão dos valores feita pelo filtro. Este filtro tem como sua base a conversão de um sistema não linear em linear.

<p style="font-family:Avantgarde; font-size:1em;"> O filtro UKF pode também ele ser aplicado em modelos não lineares, sendo mais precisos que o EKF uma vez que neste geramos pontos e os seus respetivos pesos, conseguindo aproximar-se mais da realidade uma vez que sofrem as transformações que o modelo real sofre.
</p>

<div style="widelta_th: 100%; overflow: hidden;">
    <div style="display: block;margin-left: auto;margin-right: auto; widelta_th: 100%;"><img src="./ekf_501.jpeg"  /></div>
</div>

<div style="widelta_th: 100%; overflow: hidden;">
    <div style="display: block;margin-left: auto;margin-right: auto; widelta_th: 100%;"><img src="./ukf501.jpeg"  /></div>
</div>

<p style="font-family:Avantgarde; font-size:1.5em;"> 8. Análise do Erro </p>

In [26]:
fig = go.Figure()

# EKF
trace_observation_ekf = go.Scatter(x=hz_ekf[:, 0], y=hz_ekf[:, 1], mode='markers', name='Observação EKF', marker=dict(color='magenta', size=8))
trace_true_ekf = go.Scatter(x=hxtrue_ekf[0, :].flatten(), y=hxtrue_ekf[1, :].flatten(), mode='lines', name='Verdadeiro EKF', line=dict(color='limegreen'))
trace_dead_reckoning_ekf = go.Scatter(x=hxdr_ekf[0, :].flatten(), y=hxdr_ekf[1, :].flatten(), mode='lines', name='Dead Reckoning EKF', line=dict(color='black'))
trace_ekf_prediction_ekf = go.Scatter(x=hxpred_ekf[0, :].flatten(), y=hxpred_ekf[1, :].flatten(), mode='lines', name='Predição EKF', line=dict(color='red'))

fig.add_trace(trace_observation_ekf)
fig.add_trace(trace_true_ekf)
fig.add_trace(trace_dead_reckoning_ekf)
fig.add_trace(trace_ekf_prediction_ekf)

# UKF
trace_observation_ukf = go.Scatter(x=hz_ukf[0, :], y=hz_ukf[1, :], mode='markers', name='Observação UKF', marker=dict(color='azure', size=8))
trace_true_ukf = go.Scatter(x=hxTrue_ukf[0, :].flatten(), y=hxTrue_ukf[1, :].flatten(), mode='lines', name='Verdadeiro UKF', line=dict(color='orange'))
trace_dead_reckoning_ukf = go.Scatter(x=hxDR_ukf[0, :].flatten(), y=hxDR_ukf[1, :].flatten(), mode='lines', name='Dead Reckoning UKF', line=dict(color='grey'))
trace_ukf_prediction_ukf = go.Scatter(x=hxEst_ukf[0, :].flatten(), y=hxEst_ukf[1, :].flatten(), mode='lines', name='Predição UKF', line=dict(color='brown'))

fig.add_trace(trace_observation_ukf)
fig.add_trace(trace_true_ukf)
fig.add_trace(trace_dead_reckoning_ukf)
fig.add_trace(trace_ukf_prediction_ukf)

fig.update_layout(
    height=600,
    width=800,
    title_text='Comparação das Simulações do EKF e do UKF',
    xaxis_title='x',
    yaxis_title='y',
)

fig.show()

<p style="font-family:Avantgarde; font-size:1em;"> Decidimos que para observar melhor as diferentes linhas e diferentes traços que compõem os dois gráficos anteriores juntá-los num só. No gráfico acima representado conseguimos observar algumas diferenças relativamente aos traços e, conseguimos também retirar algumas conclusões. Tais como, o desempenho do Dead Reckoning em ambos ser o mais diferente e afastado de todas as outras linhas e aglomerados de pontos. Para podermos tirar conclusões decidimos representar os vários erros num gráfico de linhas de modo a conseguirmos observar todas ao mesmo tempo. Nesse gráfico está representado o erro no eixo do y pelas iterações realizadas à aplicação dos filtros no eixo x.</p>

In [27]:
xTrue = hxtrue_ekf[0, :]
yTrue = hxtrue_ekf[1, :]

xEstEkf = hxpred_ekf[0, :]
yEstEkf = hxpred_ekf[1, :]

xEstUkf = hxEst_ukf[0, :]
yEstUkf = hxEst_ukf[1, :]

xDREkf = hxdr_ekf[0, :]
yDREkf = hxdr_ekf[1, :]

xDRUkf = hxDR_ukf[0, :]
yDRUkf = hxDR_ukf[1, :]


#done

difx = xTrue - xEstEkf
dify = yTrue - yEstEkf
error_Ekf = np.sqrt(difx ** 2 + dify ** 2)
sqrt_error_Ekf = np.sqrt(error_Ekf)

difx = xTrue - xEstUkf
dify = yTrue - yEstUkf
error_Ukf = np.sqrt(difx ** 2 + dify ** 2)
sqrt_error_Ukf = np.sqrt(error_Ukf)

difx = xTrue - xDREkf
dify = yTrue - yDREkf
error_DREkf = np.sqrt(difx ** 2 + dify ** 2)
sqrt_error_DREkf = np.sqrt(error_DREkf)

difx = xTrue - xDRUkf
dify = yTrue - yDRUkf
error_DRUkf = np.sqrt(difx ** 2 + dify ** 2)
sqrt_error_DRUkf = np.sqrt(error_DRUkf)

mean_Ekf = np.mean(error_Ekf)
mean_Ukf = np.mean(error_Ukf)

mean_DREkf = np.mean(error_DREkf)
mean_DRUkf = np.mean(error_DRUkf)

print('Mean Error DREKF:', mean_DREkf)
print('Mean Error DRUKF:', mean_DRUkf)

print('Mean Error EKF:', mean_Ekf)
print('Mean Error UKF:', mean_Ukf)

x = np.arange(error_Ekf.shape[0])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, y=error_Ekf,
    name='Error EKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=error_Ukf,
    name='Error UKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=np.full(x.shape[0],mean_Ekf),
    name='Mean error EKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=np.full(x.shape[0],mean_Ukf),
    name='Mean error UKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=error_DREkf,
    name='Error DREKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=error_DRUkf,
    name='Error DRUKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=np.full(x.shape[0],mean_DREkf),
    name='Mean error DREKF',
    mode='lines'
))

fig.add_trace(go.Scatter(
    x=x, y=np.full(x.shape[0],mean_DRUkf),
    name='Mean error DRUKF',
    mode='lines'
))

fig.update_layout(
    title_text='Erros Absolutos por iteração calculados com a Distância Euclidiana',
    xaxis_title='Iteração',
    yaxis_title='Erro absoluto',
)

fig.show()

Mean Error DREKF: 4.2469168447628345
Mean Error DRUKF: 3.895076015106975
Mean Error EKF: 0.17438322144311805
Mean Error UKF: 0.25837122971530485


<p style="font-family:Avantgarde; font-size:1em;"> Ao recorrer ao gráfico acima, a conclusão que teoricamente teríamos de tirar, relembrando a mesma esta seria que o menor erro será do UKF em seguida do EKF e só depois o Dead Reckogning uma vez que como o UKF recorre aos sigma points devia ser o que mais se aproxima da realidade uma vez que evita a derivação das matrizes Jacobianas. Ao realizar as várias iterações para cálculo do erro continuamos a verificar esta situação. Não conseguindo ter tempo para descobrir a raíz deste erro que poderá ser um erro de implementação não identificado.</p>